<a href="https://colab.research.google.com/github/manasdeshpande125/da6401_assignment1/blob/main/DL_ASG1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import fashion_mnist
import numpy as np
from  matplotlib import pyplot as plt
import time
import math
from sklearn.model_selection import train_test_split
import wandb

In [2]:
dataset= fashion_mnist.load_data()
(X_train_and_validation, y_train_and_validation), (X_test, y_test) = dataset
X_train, X_validation, y_train, y_validation = train_test_split(X_train_and_validation, y_train_and_validation, test_size=0.1, random_state=42)
X_train = (X_train/255.0).astype(np.float32)
X_validation = (X_validation/255.0).astype(np.float32)
X_test = (X_test/255.0).astype(np.float32)

print("Train Dataset Shape: ", X_train.shape)
print("Train Target Vector Shape: ", y_train.shape)
print("Test Dataset Shape:", X_test.shape)
print("Test Target Vector Shape", y_test.shape)
print("Validation Dataset Shape:", X_validation.shape)
print("Validation Target Vector Shape", y_validation.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train Dataset Shape:  (54000, 28, 28)
Train Target Vector Shape:  (54000,)
Test Dataset Shape: (10000, 28, 28)
Test Target Vector Shape (10000,)
Validation Dataset Shape: (6000, 28, 28)
Validation Target Vector Shape (6000,)


In [3]:
X_train = np.array(X_train.reshape(X_train.shape[0], 784,1))
X_test = np.array(X_test.reshape(X_test.shape[0], 784,1))
X_validation = np.array(X_validation.reshape(X_validation.shape[0], 784,1))

In [4]:
def layer_init(arr,n1,n2,init_type):
    np.random.seed(10)
    if init_type=="random":
        arr.append(np.random.randn(n1,n2)*0.1)
    elif init_type=="xavier":
        arr.append(np.random.randn(n1,n2)*np.sqrt(2/(n1+n2)))
    return arr

def param(layers,init_type):
    W=[]
    B=[]
    for i in range(len(layers)-1):
        W=layer_init(W,layers[i+1],layers[i],init_type)
        B=layer_init(B,layers[i+1],1,init_type)
    return W,B

In [5]:
#Activation function
def activation(activation_function):
    if activation_function == 'sigmoid':
        return sigmoid
    if activation_function == 'tanh':
        return tanh
    if activation_function == 'ReLU':
        return relu

def sigmoid(x, derivative = False):
    if derivative:
        return sigmoid(x)*(1-sigmoid(x))
    return 1/(1 + np.exp(-x))

def tanh(x, derivative = False):
    if derivative:
        return 1 - tanh(x)**2
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

def relu(x, derivative = False):
    if derivative:
        return (x>0)*1
    return x*(x>0)

def softmax(x,derivative = False):
    if derivative:
        return softmax(x)*(1- softmax(x))
    return np.exp(x)/np.sum(np.exp(x), axis = 0)

def one_hot(y, num_output_nodes):
    v = np.zeros((num_output_nodes, len(y)))
    for i,j in enumerate(y):
        v[j,i] = 1
    return v

In [6]:
def forward(x, W, B, activation_type):
    h = []
    a = []
    sigma = activation(activation_type)  #activation
    h.append(x)   #h0 = x
    a.append(np.dot(W[0], h[0]) + B[0])
    for i in range(len(W)-1):
        h.append(sigma(a[-1]))
        a.append(np.dot(W[i+1], h[-1]) + B[i+1])
    y_hat = softmax(a[-1])

    return y_hat, h, a

In [26]:
num_inputs_nodes = [784]
hidden_layers = [64,32,16]
num_output_nodes = [10]
init_type = "xavier"
activation_type = "sigmoid"
layers=[]
layers.extend(num_inputs_nodes)
layers.extend(hidden_layers)
layers.extend(num_output_nodes)
x = np.squeeze(X_train[5, :]).reshape(784,1)
#print(x.shape)
W_initial, B_initial = param(layers, init_type)

y_hat, h, a = forward(x, W_initial, B_initial, activation_type)

In [27]:
print(y_hat)

[[0.22847141]
 [0.10163002]
 [0.02870609]
 [0.11766904]
 [0.08940631]
 [0.05978851]
 [0.03667123]
 [0.14356312]
 [0.09937401]
 [0.09472026]]


In [31]:
for i in range(10):
  y_hat, h, a = forward(np.squeeze(X_train[i, :]).reshape(784,1) , W_initial, B_initial, activation_type)
  print("************Output for image ",i, "*************")
  print(y_hat)

************Output for image  0 *************
[[0.23010952]
 [0.10006081]
 [0.02851304]
 [0.11695792]
 [0.08902172]
 [0.06010883]
 [0.0365549 ]
 [0.14468414]
 [0.0989097 ]
 [0.09507941]]
************Output for image  1 *************
[[0.23030231]
 [0.09963766]
 [0.02872474]
 [0.11608756]
 [0.08888428]
 [0.06066985]
 [0.03745229]
 [0.14427342]
 [0.09911809]
 [0.09484981]]
************Output for image  2 *************
[[0.22897843]
 [0.10166402]
 [0.02878144]
 [0.1175915 ]
 [0.0893829 ]
 [0.05989562]
 [0.03668945]
 [0.14324597]
 [0.0990675 ]
 [0.09470316]]
************Output for image  3 *************
[[0.22966501]
 [0.098349  ]
 [0.02888103]
 [0.11681589]
 [0.08806132]
 [0.06114067]
 [0.03689665]
 [0.14525964]
 [0.09927404]
 [0.09565675]]
************Output for image  4 *************
[[0.22967552]
 [0.1005255 ]
 [0.02837747]
 [0.11650508]
 [0.08930371]
 [0.06009192]
 [0.03669014]
 [0.14483657]
 [0.09946875]
 [0.09452534]]
************Output for image  5 *************
[[0.22847141]
 [0.1